In [ ]:
# %% [markdown]
# # AQI Prediction Models Comparison
#
# This notebook contains three different approaches to predicting Air Quality Index (AQI):
# 1. Basic ARIMA Model
# 2. LSTM Neural Network with Hyperparameter Tuning
# 3. Enhanced ARIMA Model with Additional Metrics

# %% [markdown]
# ## 1. Basic ARIMA Model

# %%
import pandas as pd
from statsmodels.tsa.stattools import kpss
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
import matplotlib.pyplot as plt

# Load data
df = pd.read_csv(r'C:\Users\asus\Desktop\New folder\cleaned_AQI_data_outlier5.csv')

# Convert 'From Date' to datetime and set as index
df['From Date'] = pd.to_datetime(df['From Date'], dayfirst=True, errors='coerce')
df.set_index('From Date', inplace=True)

# Sort data by the datetime index to ensure it's in chronological order
df = df.sort_index()

# Remove any duplicate indices, keeping only the first occurrence
df = df[~df.index.duplicated(keep='first')]

# Check for invalid date entries (if any)
invalid_dates = df.index.isnull().sum()
print(f"Number of invalid date entries: {invalid_dates}")

# Ensure data frequency is set correctly, otherwise resample or fill missing data
df = df.asfreq('h')  # Changed from 'H' to 'h' to avoid deprecation warning

# Fill missing values (if any) via forward fill or interpolation
df = df.ffill()  # Changed from fillna(method='ffill') to avoid deprecation warning

# Target variable: 'Overall AQI'
aqi_series = df['Overall AQI'].dropna()

# Plot original data
plt.figure(figsize=(10, 4))
plt.plot(aqi_series, label='Original AQI Data')
plt.title('Original AQI Data')
plt.xlabel('Date')
plt.ylabel('AQI')
plt.legend()
plt.show()

# KPSS test for stationarity on original series
kpss_stat, p_value, _, _ = kpss(aqi_series, regression='c', nlags="auto")
print("KPSS Test Statistic (Original Data):", kpss_stat)
print("p-value (Original Data):", p_value)

# Apply differencing to achieve stationarity
aqi_diff = aqi_series.diff().dropna()

# KPSS test after differencing
kpss_stat_diff, p_value_diff, _, _ = kpss(aqi_diff, regression='c', nlags="auto")
print("After Differencing - KPSS Test Statistic:", kpss_stat_diff)
print("p-value after differencing:", p_value_diff)

# Plot differenced data
plt.figure(figsize=(10, 4))
plt.plot(aqi_diff, label='Differenced AQI Data')
plt.title('Differenced AQI Data (Stationary)')
plt.xlabel('Date')
plt.ylabel('AQI Difference')
plt.legend()
plt.show()

# Split into train and test sets (80-20 split)
train_size = int(len(aqi_diff) * 0.8)
train, test = aqi_diff[:train_size], aqi_diff[train_size:]

# ARIMA model (order p=1, d=1, q=1 for differenced data)
model = ARIMA(train, order=(1, 1, 1))
model_fit = model.fit()

# Forecast on test set
predictions = model_fit.forecast(steps=len(test))

# Scatter plot of actual vs predicted values
plt.figure(figsize=(10, 6))
plt.scatter(test, predictions, alpha=0.5, color='blue')
plt.plot([min(test), max(test)], [min(test), max(test)], 'r--')  # Line for perfect prediction
plt.xlabel("Actual AQI")
plt.ylabel("Predicted AQI")
plt.title("Scatter Plot of Actual vs Predicted AQI Values")
plt.show()

# Calculate evaluation metrics
mse = mean_squared_error(test, predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(test, predictions)

# Print metrics
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"Mean Absolute Error (MAE): {mae}")

# Line plot for actual vs predicted values
plt.figure(figsize=(12, 6))
plt.plot(test.index, test, label="Actual AQI", color='blue')
plt.plot(test.index, predictions, label="Predicted AQI", color='red')
plt.xlabel('Date')
plt.ylabel('AQI')
plt.title('Actual vs Predicted AQI')
plt.legend()
plt.show()

# Bar chart of evaluation metrics
metrics = {'MSE': mse, 'RMSE': rmse, 'MAE': mae}
plt.figure(figsize=(8, 5))
plt.bar(metrics.keys(), metrics.values(), color=['skyblue', 'salmon', 'lightgreen'])
plt.title('Model Evaluation Metrics')
plt.ylabel('Error Value')
plt.show()

# %% [markdown]
# ## 2. LSTM Neural Network with Hyperparameter Tuning

# %%
# Required Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras_tuner import RandomSearch
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam, RMSprop

# Load Dataset
df = pd.read_csv(r'C:\Users\asus\Desktop\New folder\cleaned_AQI_data_outlier5.csv')
data = pd.read_csv(df)

# Convert 'From Date' to datetime
data['From Date'] = pd.to_datetime(data['From Date'], dayfirst=True, errors='coerce')
data = data.dropna(subset=['From Date'])  # Drop rows with invalid dates

# Drop unnecessary columns
data = data.drop(columns=['From Date', 'Pollutant with highest AQI', 'Overall AQI Bucket'], errors='ignore')

# Remove outliers using Z-Score method
z_scores = np.abs((data - data.mean()) / data.std())
data = data[(z_scores < 3).all(axis=1)]

# Splitting Data
X = data.drop(columns=['Overall AQI'])
y = data['Overall AQI']
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.6667, random_state=42)

# Reshape data for LSTM
X_train_lstm = np.expand_dims(X_train.values, axis=-1)
X_val_lstm = np.expand_dims(X_val.values, axis=-1)

# Define a function to build the LSTM model with additional hyperparameters
def build_model(hp):
    model = Sequential()
    model.add(Input(shape=(X_train.shape[1], 1)))  # Explicitly define input shape

    # Number of LSTM layers
    num_layers = hp.Int("num_layers", 1, 3)

    for i in range(num_layers):
        model.add(LSTM(
            units=hp.Int(f"units_{i}", min_value=32, max_value=128, step=32),
            activation=hp.Choice("activation", values=["relu", "tanh"]),
            return_sequences=(i < num_layers - 1)
        ))
        model.add(Dropout(hp.Float(f"dropout_{i}", min_value=0.1, max_value=0.5, step=0.1)))

    # Dense layer
    model.add(Dense(hp.Int("dense_units", min_value=16, max_value=64, step=16), activation='relu'))
    model.add(Dense(1))

    # Optimizer
    optimizer_choice = hp.Choice("optimizer", values=["adam", "rmsprop"])
    if optimizer_choice == "adam":
        optimizer = Adam(learning_rate=hp.Float("learning_rate", min_value=1e-4, max_value=1e-2, sampling="log"))
    else:
        optimizer = RMSprop(learning_rate=hp.Float("learning_rate", min_value=1e-4, max_value=1e-2, sampling="log"))

    model.compile(optimizer=optimizer, loss="mse", metrics=["mae"])
    return model

# Hyperparameter Tuning
tuner = RandomSearch(
    build_model,
    objective="val_mae",
    max_trials=20,
    executions_per_trial=1,
    directory="lstm_hyperparameter_tuning",
    project_name="AQI_prediction_updated"
)

# Perform the search
tuner.search(
    X_train_lstm,
    y_train,
    validation_data=(X_val_lstm, y_val),
    epochs=10,
    verbose=1
)

# Get best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print("Optimal Hyperparameters:")
for key, value in best_hps.values.items():
    print(f"{key}: {value}")

# %%
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Using Best Hyperparameters
model = Sequential()
num_layers = best_hps.get('num_layers')
for i in range(num_layers):
    model.add(LSTM(
        units=best_hps.get("units_" + str(i)),
        activation=best_hps.get("activation"),
        return_sequences=(i != num_layers - 1),
        input_shape=(X_train.shape[1], 1)
    ))
    model.add(Dropout(best_hps.get("dropout_" + str(i))))
model.add(Dense(best_hps.get("dense_units"), activation='relu'))
model.add(Dense(1))
model.compile(
    optimizer=best_hps.get("optimizer"),
    loss="mse",
    metrics=["mae"]
)

# Train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(
    X_train_lstm, y_train,
    validation_data=(X_val_lstm, y_val),
    epochs=50,
    batch_size=best_hps.values.get("batch_size",32),
    callbacks=[early_stopping],
    verbose=1
)

# Evaluate the model
X_test_lstm = np.expand_dims(X_test.values, axis=-1)
# Predict on the test set
y_pred = model.predict(X_test_lstm)

# Ensure y_pred is 1-dimensional
y_pred = y_pred.flatten()

# Calculate metrics
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

# Print metrics
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"R2 Score: {r2}")
print(f"MAE: {mae}")
print(f"MAPE: {mape}")

# Import required libraries for plotting
import seaborn as sns

# Plot 1: Line Plot - Actual vs Predicted
plt.figure(figsize=(10, 6))
plt.plot(y_test.values, label="Actual AQI", color="blue", marker='o', linestyle='-')
plt.plot(y_pred, label="Predicted AQI", color="orange", marker='x', linestyle='--')
plt.xlabel("Samples")
plt.ylabel("AQI")
plt.legend()
plt.title("Line Plot: Actual vs Predicted AQI")
plt.show()

# Plot 2: Scatter Plot - Actual vs Predicted
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.7, edgecolors='k', label="Predicted vs Actual")
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color="red", linestyle='--', label="Perfect Prediction")
plt.xlabel("Actual AQI")
plt.ylabel("Predicted AQI")
plt.legend()
plt.title("Scatter Plot: Actual vs Predicted AQI")
plt.show()

# Plot 3: Bar Plot - Actual vs Predicted
index = range(len(y_test))
bar_width = 0.35
plt.figure(figsize=(10, 6))
plt.bar(index, y_test, bar_width, label="Actual AQI", color="blue", alpha=0.7)
plt.bar(np.array(index) + bar_width, y_pred, bar_width, label="Predicted AQI", color="orange", alpha=0.7)
plt.xlabel("Samples")
plt.ylabel("AQI")
plt.legend()
plt.title("Bar Plot: Actual vs Predicted AQI")
plt.show()

# Plot 4: Residual Plot - Actual vs Predicted
residuals = y_test.values - y_pred
plt.figure(figsize=(10, 6))
plt.axhline(0, color='red', linestyle='--', linewidth=1, label="Zero Residual")
plt.scatter(range(len(residuals)), residuals, color="purple", alpha=0.7, edgecolors='k', label="Residuals")
plt.xlabel("Samples")
plt.ylabel("Residuals (Actual - Predicted)")
plt.legend()
plt.title("Residual Plot")
plt.show()

# Plot 5: Histogram of Residuals
plt.figure(figsize=(10, 6))
sns.histplot(residuals, kde=True, bins=20, color="purple", alpha=0.8)
plt.axvline(0, color='red', linestyle='--', linewidth=1, label="Zero Residual")
plt.xlabel("Residuals (Actual - Predicted)")
plt.ylabel("Frequency")
plt.legend()
plt.title("Histogram of Residuals")
plt.show()

# %% [markdown]
# ## 3. Enhanced ARIMA Model with Additional Metrics

# %%
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import kpss
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import (mean_squared_error, mean_absolute_error,
                             mean_absolute_percentage_error, r2_score)

# Load data
df = pd.read_csv(r'C:\Users\asus\Desktop\New folder\cleaned_AQI_data_outlier5.csv')

# Convert 'From Date' to datetime and set as index
df['From Date'] = pd.to_datetime(df['From Date'], dayfirst=True, errors='coerce')
df.set_index('From Date', inplace=True)

# Check for invalid date entries
invalid_dates = df.index.isnull().sum()
print(f"Number of invalid date entries: {invalid_dates}")

# Set data frequency
df = df.asfreq('h')

# Drop rows with NaN values
df.dropna(inplace=True)

# Target variable: 'Overall AQI'
aqi_series = df['Overall AQI'].dropna()

# Function for KPSS test
def kpss_test(series):
    result = kpss(series, regression='c')
    print(f'KPSS Statistic: {result[0]}')
    print(f'p-value: {result[1]}')
    for key, value in result[3].items():
        print(f'Critical Values {key}: {value}')
    return result

# KPSS Test for stationarity
kpss_stat, p_value, _, _ = kpss(aqi_series, regression='c', nlags="auto")
print("KPSS Test Statistic:", kpss_stat)
print("p-value:", p_value)

# First differencing
aqi_diff = aqi_series.diff().dropna()

# KPSS test after differencing
kpss_stat_diff, p_value_diff, _, _ = kpss(aqi_diff, regression='c', nlags="auto")
print("After Differencing - KPSS Test Statistic:", kpss_stat_diff)
print("p-value after differencing:", p_value_diff)

# Plot differenced data
plt.figure(figsize=(10, 4))
plt.plot(aqi_diff, label='Differenced AQI Data')
plt.title('Differenced AQI Data (Stationary)')
plt.xlabel('Date')
plt.ylabel('AQI Difference')
plt.legend()
plt.show()

# Train-test split (80-20 split)
train_size = int(len(aqi_diff) * 0.8)
train, test = aqi_diff[:train_size], aqi_diff[train_size:]

# Build and fit ARIMA model (Change order as needed)
model = ARIMA(train, order=(1, 1, 1))  # Adjusted order
model_fit = model.fit()

# Model summary
print(model_fit.summary())

# Forecasting on test set
test_forecast = model_fit.get_forecast(steps=len(test))
test_forecast_series = pd.Series(test_forecast.predicted_mean, index=test.index)

# Calculate evaluation metrics
mse = mean_squared_error(test, test_forecast_series)
rmse = np.sqrt(mse)
mae = mean_absolute_error(test, test_forecast_series)
mape = mean_absolute_percentage_error(test, test_forecast_series) * 100
r2 = r2_score(test, test_forecast_series)

# Print evaluation metrics
print(f'RMSE: {rmse:.3f}')
print(f'MAE: {mae:.3f}')
print(f'MAPE: {mape:.3f}%')
print(f'R²: {r2:.3f}')

# Plot forecast vs actual
plt.figure(figsize=(10, 6))
plt.plot(train, label='Training Data', color='blue')
plt.plot(test, label='Actual Data', color='orange')
plt.plot(test_forecast_series, label='Forecasted Data', color='green')
plt.fill_between(test.index, test_forecast.conf_int().iloc[:, 0], test_forecast.conf_int().iloc[:, 1], color='k', alpha=0.15)
plt.title('ARIMA Model Forecast vs Actual')
plt.xlabel('Date')
plt.ylabel('PM2.5 (ug/m3)')
plt.legend()
plt.show()